In [10]:

import numpy as numpy
import matplotlib.pyplot as plt

import os
import cv2

In [56]:
# !chcp 65001    #cmd 한글로된 파일 읽어줌
# !dir

In [57]:
img = cv2.imread('data-files1/mamamoo.jpg', cv2.IMREAD_COLOR) #이미지 읽어 들이기

# print(img)
# img
cv2.imshow("image", img)    #(불러오는 창 이름, 불러오는 이미지)
# plt.imshow(img)
cv2.waitKey()
cv2.destroyAllWindows()

In [68]:
import cv2

#학습된 얼굴인식 xml 불러오기
faceCascade = cv2.CascadeClassifier("data-files1/haarcascade_frontalface_alt.xml")    

img_source = 'data-files1/mamamoo.jpg'
img = cv2.imread(img_source)

# img = cv2.imread('data-files1/mamamoo.jpg', cv2.IMREAD_COLOR) #이미지 읽어 들이기

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(25, 25),
)

for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

cv2.imshow('mamamoo',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# !pip install pytesseract
# https://github.com/UB-Mannheim/tesseract/wiki   => tesseract-ocr-w64-setup-v5.0.0-alpha.20200328.exe (64 bit) resp. 설치

# https://blog.naver.com/hn03049/221957851802 여기 참고하며 설치해 볼것!

In [101]:
# !pip install pytesseract
from PIL import Image
import pytesseract as pt 
import re
import cv2

In [99]:
# !pip install pillow

In [105]:
pt.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [109]:
filename = 'data-files1/ocr-test.jpg'
img = Image.open(filename)
# img
text = pt.image_to_string(img ,lang='kor')  #영어면 'euc'
text[:100]

'261시 빠(가41104\n\n \n\n \n\n \n\n가액                                            9126 빠0(단위:070)\n사이즈(526)      '

In [110]:
# 공백이 많기 때문에 단넘기기(\n)를 기준으로 text를 구분
size_table = re.split('\n',text)

In [111]:
matchers = ['사이즈(','어깨너비','가슴둘레','소매길이','총장']
size = [s for s in size_table if any(xs in s for xs in matchers)]

In [113]:
size

['사이즈(526)          055            066            067',
 '너       0느99                     어깨너비      37.5      38.6      39.4',
 '멀                         가슴둘레            88.3            93.6            98.4',
 '                                                소매길이             01             6012            64.5']

In [116]:
#  dataframe형식으로 만들기 위해 "정규표현식"을 사용하여 데이터를 정제
# 추출한 size를 split()를 사용하여 반복문을 통해 리스트로 붙여줍니다.

import pandas as pd
size2 = []
for i in range(len(size)):
    size2.append(size[i].split())

In [122]:
print(size2)
size[1][0]

[['사이즈(526)', '055', '066', '067'], ['너', '0느99', '어깨너비', '37.5', '38.6', '39.4'], ['멀', '가슴둘레', '88.3', '93.6', '98.4'], ['소매길이', '01', '6012', '64.5']]


'너'

In [123]:
if '총장' or '소매길이' or '어깨너비' or '가슴둘레' not in size2[1][0] + size2[2][0]+size2[3][0]+size2[4][0]:
    pass
else:
    match = re.match(r"([가-힣]+)([(0-9].+)", size2[0][0], re.I)
    if match:
        items = match.groups()
    size2[0][0] = items[0]

    size3 = list(map(list, zip(*size2))) #리스트를 transpose하는 법
    #import numpy as np
    #np.array(a).T.tolist() - 똑같은 방법  -> array 변환 -> 전치 -> tolist
    df_size = pd.DataFrame(size3[1:], columns = size3[0])

In [128]:
len(size2)

4

In [124]:
size3=[]
for i in range(len(size2)):
    if len(size2[i]) == len(size2[0]):
        size3.append(size2[i])
    else:
        diff = len(size2[i]) - len(size2[0])
        size3.append(size2[i][diff:])

In [126]:
size3

[['사이즈(526)', '055', '066', '067'],
 ['어깨너비', '37.5', '38.6', '39.4'],
 ['가슴둘레', '88.3', '93.6', '98.4'],
 ['소매길이', '01', '6012', '64.5']]

In [129]:
size4 = list(map(list, zip(*size3))) #리스트를 transpose하는 법
size4 

[['사이즈(526)', '어깨너비', '가슴둘레', '소매길이'],
 ['055', '37.5', '88.3', '01'],
 ['066', '38.6', '93.6', '6012'],
 ['067', '39.4', '98.4', '64.5']]

In [130]:
df_size = pd.DataFrame(size4[1:], columns = size4[0])
df_size

,사이즈(526),어깨너비,가슴둘레,소매길이
0,055,37.5,88.3,01
1,066,38.6,93.6,6012
2,067,39.4,98.4,64.5
